# CAMELS Parameter Inference Diagnostics

This notebook visualises regression results by comparing predicted vs. target cosmological and astrophysical parameters.

In [ ]:
import pathlib

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

plt.style.use("seaborn-v0_8")
sns.set_context("talk")

In [ ]:
# Update this path if your CSV lives elsewhere
csv_path = pathlib.Path("outputs/test_predictions.csv").resolve()
df = pd.read_csv(csv_path)
df.head()

In [ ]:
parameter_names = [
    name.replace("target_", "")
    for name in df.columns
    if name.startswith("target_")
]

metrics = []
for name in parameter_names:
    target = df[f"target_{name}"]
    pred = df[f"pred_{name}"]
    mse = ((pred - target) ** 2).mean()
    mae = (pred - target).abs().mean()
    metrics.append({"parameter": name, "mse": mse, "mae": mae})

pd.DataFrame(metrics)

In [ ]:
n_params = len(parameter_names)
n_cols = min(3, n_params)
n_rows = (n_params + n_cols - 1) // n_cols
fig, axes = plt.subplots(n_rows, n_cols, figsize=(6 * n_cols, 6 * n_rows), squeeze=False)

for ax, name in zip(axes.flatten(), parameter_names):
    target = df[f"target_{name}"]
    pred = df[f"pred_{name}"]
    sns.scatterplot(x=target, y=pred, ax=ax, s=20)
    min_val = min(target.min(), pred.min())
    max_val = max(target.max(), pred.max())
    ax.plot([min_val, max_val], [min_val, max_val], "--", color="black", linewidth=1)
    ax.set_title(name)
    ax.set_xlabel("Target")
    ax.set_ylabel("Prediction")

# Hide unused axes if parameter count is not a multiple of n_cols
for ax in axes.flatten()[n_params:]:
    ax.axis("off")

plt.tight_layout()
plt.show()